In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os
from glob import glob
import numpy as np
import time
import xarray as xr

In [2]:
def Make_dire(file_path):
    if not os.path.isdir(file_path):
        os.makedirs(file_path)

In [3]:
Datas = glob(os.getcwd() + '/Data/*.nc.csv')


In [4]:
Data = pd.read_csv(Datas[0])
Clean_Data = Data.dropna()
Clean_Data.reset_index(inplace=True)

In [5]:
Datas = glob(os.getcwd() + '/Data/*S.csv')
Clean_Data = pd.DataFrame()
for ind, d in enumerate(Datas):
    print('{}/{}'.format(ind, len(Datas)))
    data = pd.read_csv(d)
    Clean_Data = pd.concat([Clean_Data, data], axis=0).reset_index(drop = True)

0/4
1/4
2/4
3/4


In [6]:
Var_names_X = ['thermalDriving', 'x', 'y', 'iceDraft', 'halineDriving']
X = Clean_Data[Var_names_X]
Y = Clean_Data['meltRate']
X_train = X.sample(frac = 0.8)
X_valid = X.drop(X_train.index)

Y_train = Y.loc[X_train.index]
Y_valid = Y.drop(X_train.index)
print(len(X_train), len(X_valid))

mean, std = X_train.mean(), X_train.std() 
meanY, stdY = Y_train.mean(), Y_train.std() 
maxY = max(Y_train)
X_train_N, X_valid_N = np.array((X_train - mean)/std), np.array((X_valid - mean)/std)
#Y_train_N, Y_valid_N = np.array((Y_train-meanY)/stdY), np.array((Y_valid-meanY)/stdY)
Y_train_N, Y_valid_N = np.array(Y_train / maxY), np.array(Y_valid/maxY)

7483133 1870783


In [7]:
def Model_init(shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape))
    model.add(tf.keras.layers.Dense(32, activation = 'relu'))
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dense(32, activation = 'relu'))
    model.add(tf.keras.layers.Dense(1, activation = 'relu'))
    model.compile(optimizer='adam',
                 loss = 'mse',
                metrics = ['mae', 'mse'])
    return model
def Model_init2(shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape))
    model.add(tf.keras.layers.Dense(32, activation = 'swish'))
    model.add(tf.keras.layers.Dense(64, activation = 'swish'))
    model.add(tf.keras.layers.Dense(64, activation = 'swish'))
    model.add(tf.keras.layers.Dense(32, activation = 'swish'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam',
                 loss = 'mse',
                metrics = ['mae', 'mse'])
    return model

In [8]:
Datas

['/home/bouissob/Script/Data/data_Ocean1_COM_NEMO-CNRS.csv',
 '/home/bouissob/Script/Data/data_Ocean4_COM_NEMO-CNRS.csv',
 '/home/bouissob/Script/Data/data_Ocean3_COM_NEMO-CNRS.csv',
 '/home/bouissob/Script/Data/data_Ocean2_COM_NEMO-CNRS.csv']

In [9]:
if len(Datas) ==1:
    Temp = Datas[0].split('/')
    Name = Temp[len(Temp) - 1]
    Name = Name.replace('data_', '')
    Ocean_scen = Name.split('_')[0]
    Mod_nam = Name.replace(Ocean_scen + '_', '').split('.')[0]
else:
    Names = []
    for n in Datas:
        Temp = n.split('/')
        Name = Temp[len(Temp) - 1]
        Name = Name.replace('data_', '')
        Ocean_scens = Name.split('_')[0]
        Names.append(Ocean_scens)
    Ocean_scen = '_'.join(Names)
    Mod_nam = Name.replace(Ocean_scens + '_', '').split('.')[0]
        

In [10]:
Mod_nam

'COM_NEMO-CNRS'

In [11]:
Ocean_scen

'Ocean1_Ocean4_Ocean3_Ocean2'

In [ ]:
Nbr_Epoch = 4
model = Model_init2( (len(Var_names_X), ))
Model = model.fit(X_train_N, Y_train_N,
                   epochs = Nbr_Epoch,
                   batch_size = 50,
                   validation_data = (X_valid_N, Y_valid_N))
Model_path = os.getcwd() + '/Models/' + Mod_nam + '/' + Ocean_scen + '/'

Uniq = time.time()
Name ='Ep_{}_Input_{}_{}'.format(Nbr_Epoch, len(Var_names_X), Uniq)
filepath = Model_path + '/' + Name + '/'
Make_dire(filepath)
model.save(filepath + 'model.h5')
model.evaluate(X_train, Y_train, verbose = 0)

2022-02-18 17:37:42.731912: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-18 17:37:42.732305: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-18 17:37:42.737471: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-02-18 17:37:43.070659: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-18 17:37:43.093990: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2399835000 Hz


Epoch 1/4
149663/149663 [==============================] - 259s 2ms/step - loss: 3.4140e-04 - mae: 0.0061 - mse: 3.4140e-04 - val_loss: 1.6929e-04 - val_mae: 0.0039 - val_mse: 1.6929e-04
Epoch 2/4
149663/149663 [==============================] - 259s 2ms/step - loss: 1.5191e-04 - mae: 0.0039 - mse: 1.5191e-04 - val_loss: 1.0979e-04 - val_mae: 0.0032 - val_mse: 1.0979e-04 ETA: 1:48 - loss: 1.5721e-04  - ETA: 5s - loss: 1.5216e-04 - mae: 0.0 - ETA: 4s - loss: 1.521 - ETA: 2s - - ETA: 0s - loss: 1.5192e-04 - mae: 0.0039 - mse: 1.5192e- - ETA: 0s - loss: 1.5192e-04 - mae: 0.0039 - mse: 1.519
Epoch 3/4
149663/149663 [==============================] - 259s 2ms/step - loss: 1.2380e-04 - mae: 0.0035 - mse: 1.2380e-04 - val_loss: 1.1044e-04 - val_mae: 0.0030 - val_mse: 1.1044e-04
Epoch 4/4
149642/149663 [============================>.] - ETA: 0s - loss: 1.1298e-04 - mae: 0.0034 - mse: 1.1298e-04

In [ ]:
Data = Clean_Data.loc[Clean_Data.date == 1]
X = Data[Var_names_X]
X = np.array((X - mean)/std)
X = np.array(X).reshape(-1, 5, )
Y_mod = Model.model(X)
Data['Mod_Melt'] = Y_mod.numpy().reshape(len(Y_mod, )) * maxY
#Data['Mod_Melt'] = (Y_mod.numpy().reshape(len(Y_mod, )) * stdY) + meanY
Dataset = Data.set_index(['y', 'x'])
Dataset = Dataset.to_xarray()
Dataset.Mod_Melt.plot(cmap = plt.get_cmap('viridis'))
plt.figure()
Dataset.meltRate.plot(cmap = plt.get_cmap('viridis'))

In [ ]:
mean.to_pickle(filepath + 'MeanX.pkl')
std.to_pickle(filepath + 'StdX.pkl')
np.savetxt(filepath + 'MaxY.csv', np.array(maxY).reshape(1, ))
tmx = int(Clean_Data.loc[len(Clean_Data) - 1].date)
#STD = pd.read_pickle(pwd + '/std.pkl')